<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/thermodynamics/Phase_envelopes_of_oil_and_gas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Phase enevelopes of oil and gas
#@markdown Demonstration of phase envelopes of gas and oil
#!git clone https://github.com/EvenSol/NeqSim-Colab.git
!pip install py4j
!pip install NeqSim-Colab/lib/neqsim-0.0.1-py3-none-any.whl